In [1]:
import glob
import pandas as pd
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
import astropy.units as u

pd.set_option('display.max_rows', 500)

In [2]:
class StarPlanner:
    def __init__(self, location, local_time_str, timezone_offset):
        """
        Initialize the StarPlanner instance.

        :param location: EarthLocation instance.
        :param local_time_str: Local time as a string (e.g., '2025-02-27 20:30:00').
        :param timezone_offset: Offset in hours from UTC (default is 8 hours).
        """
        self.location = location
        self.local_time_str = local_time_str

        self.time = Time(local_time_str) - timezone_offset * u.hour
        self.altaz_frame = AltAz(obstime=self.time, location=self.location)
        self.zenith_coord = None

    @staticmethod
    def read_star_file(filename):
        """
        Read a star data file into a pandas DataFrame.

        :param filename: Path to the star data file.
        :return: DataFrame with star data or None if an error occurs.
        """
        try:
            df = pd.read_csv(
                filename,
                skiprows=2,
                sep='|',
                quotechar='"',
                skipinitialspace=True,
                names=['main_id', 'ra', 'dec', 'sp_type', 'B', 'V']
            )
            return df
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            return None

    def load_star_data(self):
        """
        Load star data from files matching the specified glob pattern.

        :param pattern: Glob pattern to search for star files.
        :return: Combined DataFrame of star data.
        """
        star_files = glob.glob("*_stars.txt")
        all_stars = []
        for file in star_files:
            df = self.read_star_file(file)
            if df is not None:
                all_stars.append(df)
        if all_stars:
            combined_df = pd.concat(all_stars, ignore_index=True)
            return combined_df
        return pd.DataFrame()

    def calculate_zenith(self):
        """
        Calculate the zenith coordinate (in ICRS) for the current observer location and time.

        :return: SkyCoord instance representing the zenith in ICRS.
        """
        zenith_altaz = SkyCoord(alt=90 * u.deg, az=0 * u.deg, frame=self.altaz_frame)
        self.zenith_coord = zenith_altaz.transform_to('icrs')
        return self.zenith_coord

    def get_visible_stars(self, combined_df, max_separation=60 * u.deg):
        """
        Determine and return stars that are within a given angular separation from the zenith.
        Also calculate their altitude and azimuth in the local frames.

        :param combined_df: DataFrame containing star data.
        :param max_separation: Maximum angular separation from zenith to consider (default 60 degrees).
        :return: DataFrame with visible stars information sorted by separation.
        """
        if self.zenith_coord is None:
            self.calculate_zenith()

        stars_coords = SkyCoord(
            ra=combined_df['ra'].values * u.degree,
            dec=combined_df['dec'].values * u.degree
        )
        separations = self.zenith_coord.separation(stars_coords)
        mask = separations < max_separation

        visible_stars = combined_df[mask].copy()
        visible_stars['separation_degrees'] = separations[mask].degree

        visible_stars_coords = SkyCoord(
            ra=visible_stars['ra'].values * u.degree,
            dec=visible_stars['dec'].values * u.degree
        )
        stars_altaz = visible_stars_coords.transform_to(self.altaz_frame)
        visible_stars['altitude'] = stars_altaz.alt.degree
        visible_stars['azimuth'] = stars_altaz.az.degree

        return visible_stars.sort_values('separation_degrees')

In [3]:
def main():
    singapore = EarthLocation(
        lat=1.3521 * u.deg,  # North
        lon=103.8198 * u.deg,  # East
        height=0 * u.m
    )

    local_time = "2025-02-27 20:30:00"
    planner = StarPlanner(singapore, local_time, timezone_offset=8)
    zenith_radec = planner.calculate_zenith()

    print(
        "Zenith point in Singapore at {} (local time):".format(local_time)
    )
    print("RA: {}".format(zenith_radec.ra.to_string(unit=u.hour, sep=":")))
    print("Dec: {}".format(zenith_radec.dec.to_string(unit=u.degree, sep=":")))

    combined_df = planner.load_star_data()
    if combined_df.empty:
        print("No star data files found or no valid data.")
        return

    visible_stars = planner.get_visible_stars(combined_df)
    visible_stars = visible_stars.sort_values("separation_degrees")
    target_stars = visible_stars[(visible_stars["V"] >= 4) & (visible_stars["V"] <= 5)]
    print("\nStars within 60 degrees of zenith with altitude and azimuth and between magnitudes 4 and 5:")
    print("Total count: {}".format(len(target_stars)))

    o_spectral = {"I": [12], "III": [1], "V": [11]}
    b_spectral = {"I": [129, 97], "III": [140, 67], "V": [80, 127, 115]}
    a_spectral = {"I": [649, 648], "III": [500, 510], "V": [466, 515, 513]}
    f_spectral = {"I": [], "III": [1446, 1481], "V": [1456, 1462]}
    g_spectral = {"I": [777, 772, 780], "III": [944, 747, 745], "V": [742, 907]}
    k_spectral = {"I": [1140, 1155], "III": [1153, 1138, 1148], "V": [1095]}
    m_spectral = {"I": [968], "III": [966, 973, 969], "V": []}

    spectral_data = {
        "O": o_spectral,
        "B": b_spectral,
        "A": a_spectral,
        "F": f_spectral,
        "G": g_spectral,
        "K": k_spectral,
        "M": m_spectral,
    }
    spectral_order = ["O", "B", "A", "F", "G", "K", "M"]
    luminosity_order = ["I", "III", "V"]

    ordered_rows = []
    for sp in spectral_order:
        sp_dict = spectral_data.get(sp, {})
        for lum in luminosity_order:
            star_ids = sp_dict.get(lum, [])
            if star_ids:
                rows = target_stars.loc[star_ids]
                ordered_rows.append(rows)

    ordered_df = pd.concat(ordered_rows)
    display(ordered_df)

    ordered_df.to_csv("ordered_target_stars.csv", index=False)


In [4]:
if __name__ == "__main__":
    main()

Zenith point in Singapore at 2025-02-27 20:30:00 (local time):
RA: 5:54:20.89187937
Dec: 1:20:53.84452394

Stars within 60 degrees of zenith with altitude and azimuth and between magnitudes 4 and 5:
Total count: 296


,main_id,ra,dec,sp_type,B,V,separation_degrees,altitude,azimuth
12,* 29 CMa,109.668246,-24.558700,O7Iafpvar,4.800,4.950,33.002418,56.995326,143.224570
1,* ksi Per,59.741260,35.791030,O7.5III(n)((f)),4.080,4.060,43.608241,46.387529,325.571785
11,* 15 Mon,100.244415,9.895756,O7V+B1.5/2V,4.450,4.680,14.400955,75.597819,53.304329
129,* iot CMa,104.034269,-17.054239,B3Ib,4.297,4.385,23.888350,66.109876,141.178083
97,* chi Aur,83.181977,32.192022,B4Ia,5.150,4.790,31.261576,58.735498,351.304535
140,HD 57821,110.556372,-19.016601,B5II/III,4.893,4.950,29.673362,60.324548,134.541938
67,* del For,55.562094,-31.938362,B5III,4.814,4.973,45.664001,44.332476,220.426558
80,* tau Tau,70.561247,22.956903,B3V,4.138,4.258,27.805238,62.192101,322.490127
127,* lam CMa,97.042533,-32.580068,B4V,4.310,4.480,34.857153,55.140427,167.619156
115,* lam Col,88.278673,-33.801363,B5V,4.720,4.870,35.150851,54.846637,180.584652
